# LR Delivery Time Prediction — Final Submission

**Name:** M SRUTHI RAJ

**Assignment:** Linear Regression — Parcel Delivery Time Estimation

**Files included:** Notebook, PDF report, outputs (plots & artifacts), dataset.

---

In [ ]:
# Imports and display settings
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib, statsmodels.api as sm
sns.set_style('whitegrid')
pd.set_option('display.max_columns', 200)


## 1. Load data
Dataset `porter_data_1.csv` is included in the ZIP. The notebook recreates the preprocessing and modeling steps used for evaluation.

In [ ]:
DATA_PATH = 'porter_data_1.csv'
assert os.path.exists(DATA_PATH), "Dataset file porter_data_1.csv must be in the same folder as the notebook."
df = pd.read_csv(DATA_PATH)
print('Loaded shape:', df.shape)
df.head()

In [ ]:
# Create delivery_minutes target if required
if 'delivery_minutes' not in df.columns and {'created_at','actual_delivery_time'}.issubset(df.columns):
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'], errors='coerce')
    df['delivery_minutes'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds() / 60.0
target = 'delivery_minutes'
print('Target stats:'); display(df[target].describe())

## 2. Preprocessing & Feature Engineering
- Fill missing values
- Create time features if timestamps exist
- Encode categorical variables (label-encode market_id; one-hot small categorical cols)
- Scale numeric features

In [ ]:
# Basic preprocessing and feature creation
df = df.dropna(subset=[target]).copy()
if 'created_at' in df.columns:
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['order_hour'] = df['created_at'].dt.hour
    df['order_weekday'] = df['created_at'].dt.weekday

for c in ['market_id','store_primary_category','order_protocol']:
    if c in df.columns:
        df[c] = df[c].astype('category')

# Train-test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print('Train, Val shapes:', train_df.shape, val_df.shape)

In [ ]:
# Select core numeric features present in dataset
numeric_candidates = ['total_items','subtotal','num_distinct_items','min_item_price','max_item_price','total_onshift_dashers','total_busy_dashers','total_outstanding_orders','distance']
core_numeric = [c for c in numeric_candidates if c in train_df.columns]

# Fill numeric missing values with median
for c in core_numeric:
    med = train_df[c].median()
    train_df[c] = train_df[c].fillna(med)
    val_df[c] = val_df[c].fillna(med)

# Label-encode market_id to avoid many dummies
from sklearn.preprocessing import LabelEncoder
categorical_feats = []
if 'market_id' in train_df.columns:
    le = LabelEncoder()
    train_df['market_id_le'] = le.fit_transform(train_df['market_id'].astype(str))
    val_df['market_id_le'] = le.transform(val_df['market_id'].astype(str))
    categorical_feats.append('market_id_le')

# One-hot small categorical columns
ohe_cols = [c for c in ['store_primary_category','order_protocol'] if c in train_df.columns]
X_train = train_df[core_numeric + categorical_feats + ohe_cols].copy()
X_val = val_df[core_numeric + categorical_feats + ohe_cols].copy()
X_train = pd.get_dummies(X_train, columns=ohe_cols, drop_first=True)
X_val = pd.get_dummies(X_val, columns=ohe_cols, drop_first=True)
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

y_train = train_df[target].values
y_val = val_df[target].values

print('Feature matrix shape:', X_train.shape)

In [ ]:
# Cap numeric outliers at 1st and 99th percentiles and scale numeric features
for c in core_numeric:
    lower = train_df[c].quantile(0.01); upper = train_df[c].quantile(0.99)
    X_train[c] = X_train[c].clip(lower,upper)
    X_val[c] = X_val[c].clip(lower,upper)

scaler = StandardScaler()
if core_numeric:
    X_train[core_numeric] = scaler.fit_transform(X_train[core_numeric])
    X_val[core_numeric] = scaler.transform(X_val[core_numeric])

# Save scaler for reproducibility
import joblib
joblib.dump(scaler, 'scaler.joblib')

## 3. Baseline Linear Regression
Train a baseline OLS and evaluate on validation set.

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
y_pred_train = lr.predict(X_train); y_pred_val = lr.predict(X_val)

def metrics(y_true, y_pred):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    return {'rmse': mean_squared_error(y_true,y_pred,squared=False),
            'mae': mean_absolute_error(y_true,y_pred),
            'r2': r2_score(y_true,y_pred)}

print('Train metrics:', metrics(y_train, y_pred_train))
print('Val metrics  :', metrics(y_val, y_pred_val))

## 4. Select top features by coefficient magnitude and retrain
(Practical, fast alternative to RFE for large datasets.)

In [ ]:
# Select top 12 features by absolute coefficient from baseline model
coef_series = pd.Series(lr.coef_, index=X_train.columns).abs().sort_values(ascending=False)
topN = min(12, len(coef_series))
top_features = coef_series.head(topN).index.tolist()
print('Top features:', top_features)

# Retrain on top features
X_train_top = X_train[top_features].copy()
X_val_top = X_val[top_features].copy()
lr_top = LinearRegression().fit(X_train_top, y_train)
y_pred_val_top = lr_top.predict(X_val_top)
print('Top-features val metrics:', metrics(y_val, y_pred_val_top))

# Save final model
joblib.dump(lr_top, 'final_lr_model_top.joblib')

## 5. Interpretability & Diagnostics
- Statsmodels OLS summary for selected features
- Residual plots and QQ-plot

In [ ]:
# Statsmodels OLS
X_sm = sm.add_constant(X_train_top)
ols = sm.OLS(y_train, X_sm).fit()
print(ols.summary())

# Residual diagnostics on validation set
import matplotlib.pyplot as plt
y_val_pred = lr_top.predict(X_val_top)
residuals = y_val - y_val_pred

plt.figure(figsize=(6,4))
plt.scatter(y_val_pred, residuals, alpha=0.2); plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted'); plt.ylabel('Residuals'); plt.title('Residuals vs Predicted'); plt.show()

sm.qqplot(residuals, line='45'); plt.title('QQ-plot of residuals'); plt.show()

## 6. Save important plots and artifacts
(Plots saved to `outputs/` for inclusion in the PDF/report.)

In [ ]:
os.makedirs('outputs', exist_ok=True)
# Target distribution
plt.figure(figsize=(8,4))
sns.histplot(train_df[target], bins=60, kde=True)
plt.title('Target distribution (train)'); plt.savefig('outputs/target_distribution.png', dpi=150); plt.close()
# Correlation heatmap for numeric features
num_for_corr = core_numeric.copy()
if core_numeric:
    plt.figure(figsize=(10,8))
    corr = train_df[num_for_corr + [target]].corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation heatmap (train)'); plt.savefig('outputs/correlation_heatmap.png', dpi=150); plt.close()

# Actual vs predicted (Val)
plt.figure(figsize=(6,6))
plt.scatter(y_val, y_pred_val_top, alpha=0.2)
plt.plot([y_val.min(), y_val.max()],[y_val.min(), y_val.max()],'r--')
plt.xlabel('Actual'); plt.ylabel('Predicted'); plt.title('Actual vs Predicted (Val)'); plt.savefig('outputs/actual_vs_predicted_val.png', dpi=150); plt.close()

# Residuals and QQ already plotted above; save residuals plot and qq as files
plt.figure(figsize=(6,4))
plt.scatter(y_pred_val_top, residuals, alpha=0.2); plt.axhline(0,color='r',linestyle='--')
plt.xlabel('Predicted'); plt.ylabel('Residuals'); plt.title('Residuals vs Predicted'); plt.savefig('outputs/residuals_vs_predicted.png', dpi=150); plt.close()

import statsmodels.api as smm
fig = smm.qqplot(residuals, line='45', fit=True); fig.savefig('outputs/qqplot_residuals.png', dpi=150); plt.close(fig)

# Save feature importance table
pd.DataFrame({'feature': top_features}).to_csv('outputs/selected_features.csv', index=False)

# Save models in outputs/artifacts
os.makedirs('outputs/artifacts', exist_ok=True)
shutil.copy('final_lr_model_top.joblib', 'outputs/artifacts/final_lr_model_top.joblib')
shutil.copy('scaler.joblib', 'outputs/artifacts/scaler.joblib')

print('Saved outputs in outputs/ directory')

## 7. Conclusions & Recommendations

- Summarize model performance (RMSE/MAE/R²) and interpret top features.
- Provide operational recommendations (staffing, routing, micro-hubs) and limitations.

---

**End of notebook**